   ## COMBINANDO DATOS<a class="tocSkip">
  ## EL460 ELECTIVA I - CIENCIA DE DATOS II <a class="tocSkip">
  ### Ingenieria Electrónica <a class="tocSkip">
  ### Universidad Popular del Cesar <a class="tocSkip">
  ### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**

----

# Combinación de conjuntos de datos

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker).

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [ ]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

In [ ]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [ ]:
melb_df = pandas.read_csv('melb_data.csv')
melb_df[:3]

## Datos adicionales

Vamos a aumentar los datos presentes en el conjunto dado con un dataset similar: las publicaciones de la plataforma AirBnB en Melbourne en el año 2018. El objetivo es estimar con mayor presición el valor del vecidanrio de cada propiedad.

Para ello, utilizaremos [un conjunto de datos](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) de *scrapings* del sitio realizado por [Tyler Xie](https://www.kaggle.com/tylerx), también disponible en una competencia de Kaggle.

In [ ]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pandas.read_csv('cleansed_listings_dec18.csv',
    usecols=interesting_cols)

In [ ]:
airbnb_df[:3]

In [ ]:
len(airbnb_df)

Para poder realizar esta combinación de datos, tenemos que encontrar una columna que contenga información en común. Utilizaremos el zipcode.

* ¿Qué desventajas tiene utilizar el zipcode para agregar información sobre la vecindad de cada propiedad en venta? ¿Se corresponde con lo que entendemos intuitivamente como vecindad?
* ¿Qué otros métodos podrían utilizarse?

In [ ]:
airbnb_df.zipcode.value_counts()[:5]

 Sin embargo, este zipcode tiene tipos diferentes, que tendremos que estandarizar antes de continuar con la combinación de los datos.

In [ ]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

In [ ]:
airbnb_df.zipcode.value_counts().head(5)

### Valores en común

Antes de unir los conjuntos de datos, tenemos que asegurarnos de que tengamos suficientes registros en común como para agregar información relevantes.

In [ ]:
intersection = numpy.intersect1d(
    airbnb_df.zipcode.values, melb_df.Postcode.values, assume_unique=False)
print("Airbnb unique zipcodes", len(airbnb_df.zipcode.unique()))
print("Sales unique zipcodes", len(melb_df.Postcode.unique()))
print("Common zipcodes", len(intersection))

In [ ]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df.Postcode.isin(intersection).sum() / len(melb_df))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df.zipcode.isin(intersection).sum() / len(airbnb_df))

### Exploración visual

Para terminar de asegurarnos de que las áreas representadas por ambos datasets son consitentes, podemos representar las coordenadas en un mapa usando Plotly.

Aunque estas visualizaciones no son completas en cuanto a la información geográfica que muestran, aproximadamente podemos ver que se tratan de zonas similares.

In [ ]:
import plotly.express as px
data = melb_df.sample(300)
fig = px.scatter_geo(
    data, lat=data.Lattitude, lon=data.Longtitude, color=data.YearBuilt
    )
fig.update_geos(fitbounds="locations")
fig.show()

In [ ]:
import plotly.express as px
color_col = 'city'
data = airbnb_df[~airbnb_df[color_col].isna()].sample(300)
fig = px.scatter_geo(
    data, lat=data.latitude, lon=data.longitude, color=data[color_col]
    )
fig.update_geos(fitbounds="locations")
fig.show()

## Operación merge

Es hora de combinar ambos conjuntos de datos. Una opción frecuentemente tentadora es aplicar la operación merge sin analizar demasiado la situación. Sin embargo, ¡esta operación puede ocasionar una **explosión exponencial** de nuestro conjunto de datos!

¿Por qué la siguiente operación genera un DataFrame con 2 millones de filas? 

In [ ]:
## Incorrecto!
len(melb_df.merge(airbnb_df, how='left',
                  left_on='Postcode', right_on='zipcode'))

Recordemos que nuestro objetivo es agregar información sobre el valor del vecindario de la propiedad. ¿Qué información del conjunto de airbnb nos sirve para representar esto?

* Promedio de `price` por zipcode
* Promedio de `weekly_price` por zipcode
* Promedio de `monthly_price` por zipcode

Queda a interpretación de ustedes decidir cuál de todos ellos utilizar.

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price']

Cuando queremos agregar un único dato luego de agrupar un DataFrame, la operación es bastante sencilla. Sólo tienen que tener en cuenta que el nombre de la columnas será el mismo, aunque su valor ahora esté agregado.

In [ ]:
# Option 1: inefficient form.
# The aggregation is calculated over all columns, and after is calculated
# we filter only the columns we want.
# airbnb_df.groupby('zipcode').mean()[relevant_cols].reset_index()[:5]

# Option 2: efficient form
# First we filter out the columns we want, and then we only aggregate
# those. Be careful to include the grouping column as well.
airbnb_df[relevant_cols + ['zipcode']]\
  .groupby('zipcode').mean().reset_index()[:5]

Agrupar y agregar utilizando más de un tipo de operación es un poco más complicado. Pandas nos provee de muchas formas de escribir la misma operación. Ninguna de ellas es muy intuitiva, y usualmente requiere renombrar las columnas para darles nombres más significativos.

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()

In [ ]:
airbnb_price_by_zipcode

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']
## Option 1
# Pass a dictionary where the keys are the original columns to aggregate and
# the values are the operations (or list of operations).
airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
  .agg({'price': ['mean', 'count'], 'weekly_price': 'mean',
        'monthly_price': 'mean'})\
  .reset_index()
# Flatten the two level columns
airbnb_price_by_zipcode.columns = [
  ' '.join(col).strip()
  for col in airbnb_price_by_zipcode.columns.values]
# Rename columns
airbnb_price_by_zipcode = airbnb_price_by_zipcode.rename(
    columns={'price mean': 'airbnb_price_mean',
             'price count': 'airbnb_record_count',
             'weekly_price mean': 'airbnb_weekly_price_mean',
             'monthly_price mean': 'airbnb_monthly_price_mean'}
)

## Option 2
# Pass as argument name the new name of the column, and as value a tuple where
# the first value is the original column and the second value is the operation.
# airbnb_price_by_zipcode = airbnb_df[relevant_cols].groupby('zipcode')\
#   .agg(airbnb_record_count=('price', 'count'),
#        airbnb_price_mean=('price', 'mean'),
#        airbnb_weekly_price_mean=('weekly_price', 'mean'),
#        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
#   .reset_index()

airbnb_price_by_zipcode[:3]

Guardamos el dataset construido ya que nos sera util para el entregable

In [ ]:
airbnb_price_by_zipcode.to_csv("airbnb_price_by_zipcode.csv", index=None)
dfotro= pandas.read_csv('airbnb_price_by_zipcode.csv')

In [ ]:
#from google.colab import files
#files.download('airbnb_price_by_zipcode.csv')
dfotro.to_csv("./airbnb_price_by_zipcode.csv")

Finalmente hacemos la combinacion de ambos datasets creados

In [40]:
merged_sales_df = melb_df.merge(
    airbnb_price_by_zipcode, how='left',
    left_on='Postcode', right_on='zipcode'
)
merged_sales_df.sample(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
4103,Moonee Ponds,49 Derby St,3,h,855000.0,S,Brad,16/07/2016,6.9,3039.0,...,Moonee Valley,-37.76410,144.91120,Western Metropolitan,6232.0,3039.0,100.980769,52.0,392.500000,1533.333333
7862,Flemington,8A John St,3,h,1085000.0,SP,Nelson,22/04/2017,4.4,3031.0,...,Moonee Valley,-37.78390,144.92730,Northern Metropolitan,3593.0,3031.0,119.914894,188.0,645.388889,2184.090909
9549,Gisborne,60 Hamilton St,3,h,540000.0,SA,McDonald,17/06/2017,45.9,3437.0,...,Macedon Ranges,-37.48701,144.58567,Northern Victoria,3376.0,NaN,NaN,NaN,NaN,NaN
8722,Albion,2 Bazentin St,3,h,570000.0,S,Barry,1/07/2017,10.5,3020.0,...,Brimbank,-37.77917,144.82223,Western Metropolitan,2185.0,3020.0,69.555556,45.0,160.000000,600.000000
4742,Port Melbourne,405/155 Beach St,2,u,1610000.0,S,Kay,10/12/2016,3.8,3207.0,...,Port Phillip,-37.83920,144.92570,Southern Metropolitan,8648.0,3207.0,200.210117,257.0,1480.790698,4743.964286


In [41]:
merged_sales_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'zipcode',
       'airbnb_price_mean', 'airbnb_record_count', 'airbnb_weekly_price_mean',
       'airbnb_monthly_price_mean'],
      dtype='object')

In [43]:
merged_sales_df.to_csv("melb_data_extended.csv", index=None)
dfotro1= pandas.read_csv('melb_data_extended.csv')

In [44]:
#from google.colab import files
#files.download('melb_data_extended.csv')
dfotro1.to_csv("./melb_data_extended.csv")